In [1]:
import sys
import os
import numpy as np
import pandas as pd
import json
import pymysql
import cryptography
from datetime import datetime, date
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType, DoubleType
from hashlib import sha256, md5
from util import *
from table_config import *

24/11/20 18:35:14 WARN Utils: Your hostname, chenyideMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.31.64 instead (on interface en0)
24/11/20 18:35:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/20 18:35:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 数据处理

## Get Raw Data

In [2]:
data_source_path = '/Users/chenyi/Documents/sag/Final_Project/code/data_deploy/chatunitest-info'
# 找到所有.json文件并存储在列表中
class_json_files = [os.path.join(dirpath, file)
              for dirpath, dirnames, files in os.walk(data_source_path)
              for file in files if file.endswith('class.json')]

method_json_files = [os.path.join(dirpath, file)
              for dirpath, dirnames, files in os.walk(data_source_path)
              for file in files if file.endswith('.json') and not file.endswith('class.json')]

In [5]:
# 存储所有DataFrame的列表
class_df_list = []
method_df_list = []

for json_file in class_json_files:
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        df = pd.DataFrame([data])
        class_df_list.append(df)

for json_file in method_json_files:
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        df = pd.DataFrame([data])
        method_df_list.append(df)

class_df = pd.concat(class_df_list, ignore_index=True)
method_df = pd.concat(method_df_list, ignore_index=True)

# 设置预定义类型
for key, dtype in expected_types.items():
    if key in class_df.columns:
        class_df[key] = class_df[key].astype(dtype)
    if key in method_df.columns:
        method_df[key] = method_df[key].astype(dtype)

In [6]:
class_df.drop_duplicates(subset=['fullClassName'], keep='last', inplace=True)
method_df.drop_duplicates(subset=['packageName', 'className', 'methodSignature'], keep='last', inplace=True)

In [10]:
# 转换spark DataFrame 并写入mysql表

sp_df1 = convert_empty_list_to_NULL(spark.createDataFrame(class_df))
# push_to_mysql(sp_df1.rdd, 'class_raw_data', class_table_columns)

sp_df2 = convert_empty_list_to_NULL(spark.createDataFrame(method_df))
# push_to_mysql(sp_df2.rdd, 'method_raw_data', method_table_columns)

24/11/20 18:36:41 WARN TaskSetManager: Stage 1 contains a task of very large size (1077 KiB). The maximum recommended task size is 1000 KiB.


## Class Raw Data

In [11]:
sp_df1.toPandas()

24/11/20 18:37:31 WARN TaskSetManager: Stage 3 contains a task of very large size (1077 KiB). The maximum recommended task size is 1000 KiB.


,projectName,moduleName,packageName,fullClassName,className,index,modifier,extend,implement,packageDeclaration,...,methodsBrief,constructorSigs,constructorBrief,getterSetterSigs,getterSetterBrief,constructorDeps,compilationUnitCode,classDeclarationCode,subClasses,lineCount
0,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatRobotController,ChatRobotController,17,[public ],[],[],package org.mvnsearch.chatgpt.demo;,...,"['@PostMapping(""/chat"")\n public Mono<Strin...",[],[],[],[],{},package org.mvnsearch.chatgpt.demo;\n\nimport ...,@RestController\npublic class ChatRobotControl...,[],22
1,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo.service,org.mvnsearch.chatgpt.demo.service.GPTFunctions,GPTFunctions,20,[public ],[],[GPTFunctionsStub],package org.mvnsearch.chatgpt.demo.service;,...,"['@GPTFunction(name = ""compile_java"", value = ...",[],[],[],[],{},package org.mvnsearch.chatgpt.demo.service;\n\...,@Component\npublic class GPTFunctions implemen...,[],25
2,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatGPTDemoApp,ChatGPTDemoApp,18,[public ],[],[],package org.mvnsearch.chatgpt.demo;,...,['public static void main(String[] args){}'],[],[],[],[],{},package org.mvnsearch.chatgpt.demo;\n\nimport ...,@SpringBootApplication\npublic class ChatGPTDe...,[],7
3,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ProjectBootBaseTest,ProjectBootBaseTest,19,"[public , abstract ]",[],[],package org.mvnsearch.chatgpt.demo;,...,[],[],[],[],[],{},package org.mvnsearch.chatgpt.demo;\n\nimport ...,@SpringBootTest(classes = ChatGPTDemoApp.class...,[],3
4,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.model,org.mvnsearch.chatgpt.model.ChatCompletionRequ...,ChatCompletionRequestTest,21,[public ],[],[],package org.mvnsearch.chatgpt.model;,...,['@Test\n public void testToJson() throws E...,[],[],[],[],{},package org.mvnsearch.chatgpt.model;\n\nimport...,public class ChatCompletionRequestTest {\n\n ...,[],8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.UserObjec...,UserObjectNotFoundException,72,[public ],[RuntimeException],[],package com.selimhorri.app.exception.wrapper;,...,"['public UserObjectNotFoundException(){}', 'pu...","['UserObjectNotFoundException()', 'UserObjectN...","['public UserObjectNotFoundException(){}', 'pu...",[],[],"{'java.lang.String': [], 'java.lang.Throwable'...",package com.selimhorri.app.exception.wrapper;\...,public class UserObjectNotFoundException exten...,[],22
970,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Verificat...,VerificationTokenNotFoundException,70,[public ],[RuntimeException],[],package com.selimhorri.app.exception.wrapper;,...,['public VerificationTokenNotFoundException(){...,"['VerificationTokenNotFoundException()', 'Veri...",['public VerificationTokenNotFoundException(){...,[],[],"{'java.lang.String': [], 'java.lang.Throwable'...",package com.selimhorri.app.exception.wrapper;\...,public class VerificationTokenNotFoundExceptio...,[],22
971,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Credentia...,CredentialNotFoundException,69,[public ],[RuntimeException],[],package com.selimhorri.app.exception.wrapper;,...,"['public CredentialNotFoundException(){}', 'pu...","['CredentialNotFoundException()', 'CredentialN...","['public CredentialNotFoundException(){}', 'pu...",[],[],"{'java.lang.String': [], 'java.lang.Throwable'...",package com.selimhorri.app.exception.wrapper;\..

## Method Raw Data

In [12]:
sp_df2.toPandas()

,projectName,moduleName,packageName,fullClassName,className,methodName,returnType,brief,methodSignature,sourceCode,...,isBoolean,isAbstract,isTest,parameters,dependentMethods,full_method_info,method_comment,method_annotation,lineCount,branchCount
0,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatRobotController,ChatRobotController,chat,java.lang.String,"@PostMapping(""/chat"")\n public Mono<String>...",chat(String),"@PostMapping(""/chat"")\n public Mono<String>...",...,False,False,False,['java.lang.String'],"{'java.lang.String': [], 'reactor.core.publish...","@PostMapping(""/chat"")\npublic Mono<String> cha...",,"@PostMapping(""/chat"")\n",4,1
1,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatRobotController,ChatRobotController,streamChat,java.lang.String,"@GetMapping(""/stream-chat"")\n public Flux<S...",streamChat(String),"@GetMapping(""/stream-chat"")\n public Flux<S...",...,False,False,False,['java.lang.String'],"{'java.lang.String': [], 'reactor.core.publish...",/**\n * stream chat by server-sent events\n *\...,\n * stream chat by server-sent events\n ...,"@GetMapping(""/stream-chat"")\n",4,1
2,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatGPTDemoApp,ChatGPTDemoApp,main,void,public static void main(String[] args){},main(String[]),public static void main(String[] args) {\n ...,...,False,False,False,['java.lang.String'],"{'java.lang.String': [], 'org.springframework....",public static void main(String[] args) {\n ...,,,2,1
3,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.model,org.mvnsearch.chatgpt.model.ChatCompletionRequ...,ChatCompletionRequestTest,testToJson,void,@Test\n public void testToJson() throws Exc...,testToJson(),@Test\n public void testToJson() throws Exc...,...,False,False,True,[],{'org.mvnsearch.chatgpt.model.ChatCompletionRe...,@Test\npublic void testToJson() throws Excepti...,,@Test\n,5,1
4,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.model,org.mvnsearch.chatgpt.model.ChatCompletionUsage,ChatCompletionUsage,getPromptTokens,java.lang.Integer,public Integer getPromptTokens(){},getPromptTokens(),public Integer getPromptTokens() {\n re...,...,False,False,False,[],{},public Integer getPromptTokens() {\n return...,,,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Verificat...,VerificationTokenNotFoundException,VerificationTokenNotFoundException,,public VerificationTokenNotFoundException(Thro...,VerificationTokenNotFoundException(Throwable),public VerificationTokenNotFoundException(Thro...,...,False,False,False,['java.lang.Throwable'],{'java.lang.Throwable': []},public VerificationTokenNotFoundException(Thro...,,,2,1
3579,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Credentia...,CredentialNotFoundException,CredentialNotFoundException,,public CredentialNotFoundException(){},CredentialNotFoundException(),public CredentialNotFoundException() {\n\t\tsu...,...,False,False,False,[],{},public CredentialNotFoundException() {\n su...,,,2,1
3580,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Credentia...,CredentialNotFoundException,CredentialNotFoundException,,public CredentialNotFoundException(String mess...,"CredentialNotFoundException(String, Throwable)",public CredentialNotFoundException(String mess...,...,False,False,False,"['java.lang.String', 'java.lang.Throwable']","{'java.lang.String': [], 'java.lang.Throwable'...",public CredentialNotFoundException(String me

## Process RAW Data

### 计算 Hash (不使用)

In [13]:
df1 = sp_df1.toPandas()
df2 = sp_df2.toPandas()

24/11/20 18:37:34 WARN TaskSetManager: Stage 5 contains a task of very large size (1077 KiB). The maximum recommended task size is 1000 KiB.


In [16]:
df1_hash = df1
df2_hash = df2

In [17]:
df1_hash = df1_hash.drop('index', axis=1)
df1_hash = df1_hash.reset_index()
df1_hash['import_count'] = df1_hash['imports'].map(len)
df1_hash['field_count'] = df1_hash['fields'].map(len)
df1_hash['super_class_count'] = df1_hash['superClasses'].map(len)
df1_hash['implement_count'] = df1_hash['implementedTypes'].map(len)
df1_hash['method_count'] = df1_hash['methodSigs'].map(len)
df1_hash['constructor_count'] = df1_hash['constructorSigs'].map(len)
df1_hash['constructor_dep_count'] = df1_hash['constructorDeps'].map(len)
df1_hash['sub_class_count'] = df1_hash['subClasses'].map(len)
df1_hash

,index,projectName,moduleName,packageName,fullClassName,className,modifier,extend,implement,packageDeclaration,...,subClasses,lineCount,import_count,field_count,super_class_count,implement_count,method_count,constructor_count,constructor_dep_count,sub_class_count
0,0,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatRobotController,ChatRobotController,[public ],[],[],package org.mvnsearch.chatgpt.demo;,...,[],22,383,58,2,2,48,2,2,2
1,1,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo.service,org.mvnsearch.chatgpt.demo.service.GPTFunctions,GPTFunctions,[public ],[],[GPTFunctionsStub],package org.mvnsearch.chatgpt.demo.service;,...,[],25,299,2,2,20,76,2,2,2
2,2,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatGPTDemoApp,ChatGPTDemoApp,[public ],[],[],package org.mvnsearch.chatgpt.demo;,...,[],7,126,2,2,2,23,2,2,2
3,3,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ProjectBootBaseTest,ProjectBootBaseTest,"[public , abstract ]",[],[],package org.mvnsearch.chatgpt.demo;,...,[],3,64,2,2,2,2,2,2,2
4,4,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.model,org.mvnsearch.chatgpt.model.ChatCompletionRequ...,ChatCompletionRequestTest,[public ],[],[],package org.mvnsearch.chatgpt.model;,...,[],8,103,2,2,2,21,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,969,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.UserObjec...,UserObjectNotFoundException,[public ],[RuntimeException],[],package com.selimhorri.app.exception.wrapper;,...,[],22,2,52,20,2,184,164,51,2
970,970,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Verificat...,VerificationTokenNotFoundException,[public ],[RuntimeException],[],package com.selimhorri.app.exception.wrapper;,...,[],22,2,52,20,2,212,192,51,2
971,971,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Credentia...,CredentialNotFoundException,[public ],[RuntimeException],[],package com.selimhorri.app.exception.wrapper;,...,[],22,2,52,20,2,184,164,51,2
972,972,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.payload,com.selimhorri.app.exception.payload.ExceptionMsg,ExceptionMsg,"[public , final ]",[],[Serializable],package com.selimhorri.app.exception.payload;,...,[],19,673,377,2,16,2,2,2,2


In [18]:
df2_hash = df2_hash.reset_index()
df2_hash['parameter_count'] = df2_hash['parameters'].map(len)
df2_hash['method_dep_count'] = df2_hash['dependentMethods'].map(len)
df2_hash

,index,projectName,moduleName,packageName,fullClassName,className,methodName,returnType,brief,methodSignature,...,isTest,parameters,dependentMethods,full_method_info,method_comment,method_annotation,lineCount,branchCount,parameter_count,method_dep_count
0,0,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatRobotController,ChatRobotController,chat,java.lang.String,"@PostMapping(""/chat"")\n public Mono<String>...",chat(String),...,False,['java.lang.String'],"{'java.lang.String': [], 'reactor.core.publish...","@PostMapping(""/chat"")\npublic Mono<String> cha...",,"@PostMapping(""/chat"")\n",4,1,20,255
1,1,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatRobotController,ChatRobotController,streamChat,java.lang.String,"@GetMapping(""/stream-chat"")\n public Flux<S...",streamChat(String),...,False,['java.lang.String'],"{'java.lang.String': [], 'reactor.core.publish...",/**\n * stream chat by server-sent events\n *\...,\n * stream chat by server-sent events\n ...,"@GetMapping(""/stream-chat"")\n",4,1,20,257
2,2,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.demo,org.mvnsearch.chatgpt.demo.ChatGPTDemoApp,ChatGPTDemoApp,main,void,public static void main(String[] args){},main(String[]),...,False,['java.lang.String'],"{'java.lang.String': [], 'org.springframework....",public static void main(String[] args) {\n ...,,,2,1,20,99
3,3,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.model,org.mvnsearch.chatgpt.model.ChatCompletionRequ...,ChatCompletionRequestTest,testToJson,void,@Test\n public void testToJson() throws Exc...,testToJson(),...,True,[],{'org.mvnsearch.chatgpt.model.ChatCompletionRe...,@Test\npublic void testToJson() throws Excepti...,,@Test\n,5,1,2,304
4,4,chatgpt-spring-boot-starter,chatgpt-spring-boot-starter,org.mvnsearch.chatgpt.model,org.mvnsearch.chatgpt.model.ChatCompletionUsage,ChatCompletionUsage,getPromptTokens,java.lang.Integer,public Integer getPromptTokens(){},getPromptTokens(),...,False,[],{},public Integer getPromptTokens() {\n return...,,,2,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578,3578,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Verificat...,VerificationTokenNotFoundException,VerificationTokenNotFoundException,,public VerificationTokenNotFoundException(Thro...,VerificationTokenNotFoundException(Throwable),...,False,['java.lang.Throwable'],{'java.lang.Throwable': []},public VerificationTokenNotFoundException(Thro...,,,2,1,23,27
3579,3579,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Credentia...,CredentialNotFoundException,CredentialNotFoundException,,public CredentialNotFoundException(){},CredentialNotFoundException(),...,False,[],{},public CredentialNotFoundException() {\n su...,,,2,1,2,2
3580,3580,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Credentia...,CredentialNotFoundException,CredentialNotFoundException,,public CredentialNotFoundException(String mess...,"CredentialNotFoundException(String, Throwable)",...,False,"['java.lang.String', 'java.lang.Throwable']","{'java.lang.String': [], 'java.lang.Throwable'...",public CredentialNotFoundException(String mess...,,,2,1,43,51
3581,3581,ecommerce-microservice-backend,ecommerce-microservice-backend.proxy-client,com.selimhorri.app.exception.wrapper,com.selimhorri.app.exception.wrapper.Credentia...,CredentialNotFoundException,CredentialNotFoundException,,public CredentialNotFoundException(String mess...,CredentialNotFoundException(String),...,False,['java.lang.String'],{'java.lang.String': []},public CredentialNotFoun

In [19]:
# 转换spark DataFrame 并写入mysql表

new_sp_df1 = convert_empty_list_to_NULL(spark.createDataFrame(df1_hash))
# push_to_mysql(new_sp_df1.rdd, 'class_processed_data', class_processed_table_columns)

new_sp_df2 = convert_empty_list_to_NULL(spark.createDataFrame(df2_hash))
# push_to_mysql(new_sp_df2.rdd, 'method_processed_data', method_processed_table_columns)

# 构图

## method - rely - class, method - rely - method

In [32]:
edge_index_method_rely_class_1 = match_dict(df2_hash['index'].tolist(), df2_hash, df1_hash, 'dependentMethods', 'fullClassName')
edge_index_method_rely_class_2 = match_list(df2_hash['index'].tolist(), df2_hash, df1_hash, 'parameters', 'fullClassName')
edge_index_method_rely_class = np.concatenate((edge_index_method_rely_class_1, edge_index_method_rely_class_2),axis=1)
edge_index_method_rely_class = np.unique(edge_index_method_rely_class, axis=1)

edge_index_method_rely_method = match_dict(df2_hash['index'].tolist(), df2_hash, df2_hash, 'dependentMethods', 'methodSignature', False)

In [33]:
print(edge_index_method_rely_class.shape)
print(edge_index_method_rely_method.shape)

(2, 470)
(2, 265)


## class - rely - class, class - rely - method, class - contain - method

In [34]:
# edge_index_class_rely_method = match_dict(df1_hash['index'].tolist(), df1_hash, df2_hash, 'constructorDeps', 'fullClassName', False) # no use
edge_index_class_contain_method = match_dict(df1_hash['index'].tolist(), df1_hash, df2_hash, 'methodSigs', 'methodSignature') # methodSigs包含了构造函数

edge_index_class_rely_class_1 = match_dict(df1_hash['index'].tolist(), df1_hash, df1_hash, 'constructorDeps', 'fullClassName')
edge_index_class_rely_class_2 = match_list(df1_hash['index'].tolist(), df1_hash, df1_hash, 'importTypes', 'fullClassName')
edge_index_class_method_rely_class = build_class_to_class(edge_index_class_contain_method, edge_index_method_rely_class)

In [35]:
# Test Node
target_class_node = new_sp_df1.where('isTest == 1').distinct()
target_method_node = new_sp_df2.where('isTest == 1').distinct()

target_class_node_list = target_class_node.toPandas()['index'].tolist()
target_method_node_list = target_method_node.toPandas()['index'].tolist()

# Source Class Node - Test list pair
test_list, class_list = find_class_by_test(df1_hash, target_class_node_list)
edge_index_class_rely_class = np.concatenate((np.array([test_list, class_list]), edge_index_class_rely_class_1, edge_index_class_rely_class_2, edge_index_class_method_rely_class),axis=1)
edge_index_class_rely_class = np.unique(edge_index_class_rely_class, axis=1)

In [36]:
# print(edge_index_class_rely_method.shape)
print(edge_index_class_contain_method.shape)
print(edge_index_class_method_rely_class.shape)
print(edge_index_class_rely_class.shape)

(2, 492)
(2, 120)
(2, 330)


## Edge Data: class-class, class-method, method-class, method-method

In [18]:
m2m_sp_df = convert_empty_list_to_NULL(spark.createDataFrame(edge_index_append(edge_index_method_rely_method, 'method_to_method')))
m2c_sp_df = convert_empty_list_to_NULL(spark.createDataFrame(edge_index_append(edge_index_method_rely_class, 'method_to_class')))
c2m_sp_df = convert_empty_list_to_NULL(spark.createDataFrame(edge_index_append(edge_index_class_contain_method, 'class_contain_method')))
c2c_sp_df = convert_empty_list_to_NULL(spark.createDataFrame(edge_index_append(edge_index_class_rely_class, 'class_to_class')))

# push_to_mysql(m2m_sp_df.rdd, 'edge_method_to_method', edge_index_table_columns)
# push_to_mysql(m2c_sp_df.rdd, 'edge_method_to_class', edge_index_table_columns)
# push_to_mysql(c2m_sp_df.rdd, 'edge_class_to_method', edge_index_table_columns)
# push_to_mysql(c2c_sp_df.rdd, 'edge_class_to_class', edge_index_table_columns)

In [20]:
sql = '''
CREATE TABLE IF NOT EXISTS edge_data
(
    src_id BIGINT,
    dst_id BIGINT,
    weight FLOAT,
    edge_type VARCHAR(255)
)
PARTITION BY LIST COLUMNS(edge_type)(
    PARTITION class_to_class VALUES IN ('class_to_class'),
    PARTITION class_to_method VALUES IN ('class_contain_method'),
    PARTITION method_to_class VALUES IN ('method_to_class'),
    PARTITION method_to_method VALUES IN ('method_to_method')
    );

INSERT INTO edge_data
SELECT src_id, dst_id, weight, type FROM edge_class_to_class
UNION ALL
SELECT src_id, dst_id, weight, type FROM edge_class_to_method
UNION ALL
SELECT src_id, dst_id, weight, type FROM edge_method_to_class
UNION ALL
SELECT src_id, dst_id, weight, type FROM edge_method_to_method;
'''

In [21]:
sql = '''
select * from edge_data
'''
# sp_res = pull_data(sql)

# c2c_sp_df.select('src_id', 'dst_id').toPandas()
# sp_res.select('src_id', 'dst_id').where(sp_res['edge_type']=='class_to_class').toPandas()

# TODO: 对所有三方依赖库的类名构建节点